# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [36]:
#Libraries
import pandas as pd
import numpy as np

from scipy.stats import randint, uniform

import joblib 

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeClassifier

#ussed for regression tasks
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# For classification tasks
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [37]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [38]:
df.dropna(inplace=True)

In [39]:
def clean_name(col):
    df[col] = df[col].astype(str).str.strip().str[0]
    return df[col] 

clean_name("Cabin")

0       B
1       F
2       A
3       A
4       F
       ..
8688    A
8689    G
8690    G
8691    E
8692    E
Name: Cabin, Length: 6606, dtype: object

In [40]:
df.drop(columns=['PassengerId', 'Name'], inplace=True)
df['home_planet'] = df['HomePlanet'].map({'Mars': 0, 'Europa': 1, 'Earth': 2}).fillna(0).astype(int)
df['cryo_sleep'] = df['CryoSleep'].map({False: 0, True: 1})
df['cabin'] = df['Cabin'].map({'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'T': 7})
df['destination'] = df['Destination'].map({'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e':2}).fillna(0).astype(int)
df['vip'] = df['VIP'].map({False: 0, True: 1})
df['transported'] = df['Transported'].map({False: 0, True: 1})

In [41]:
dfv1 = df.copy()

In [42]:
dfv1.drop(columns=['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Transported'], inplace=True)

In [43]:
dfv1.columns = dfv1.columns.str.lower()

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [44]:
features = dfv1.drop(columns = ["transported"])
target = dfv1["transported"] 

In [45]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

In [46]:
normalizer = StandardScaler()
normalizer.fit(X_train)

StandardScaler()

In [47]:
X_train_norm = normalizer.transform(X_train)
X_test_norm = normalizer.transform(X_test)

In [48]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [49]:
model_rf = RandomForestClassifier(max_depth=10, random_state=42)

In [50]:
model_rf.fit(X_train_norm, y_train)

RandomForestClassifier(max_depth=10, random_state=42)

In [51]:
#your code here
y_pred = model_rf.predict(X_test)

- Evaluate your model

In [52]:
#your code here
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.50      0.99      0.66       653
           1       0.74      0.03      0.06       669

    accuracy                           0.50      1322
   macro avg       0.62      0.51      0.36      1322
weighted avg       0.62      0.50      0.36      1322



In [53]:
y_pred = model_rf.predict(X_test_norm)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

Accuracy: 0.8086232980332829
F1 Score: 0.8118959107806691


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [54]:
#your code here
param_distributions = {
    'n_estimators': randint(100, 1000),              # Number of trees in the forest
    'max_depth': randint(5, 50),                     # Maximum depth of each tree
    'min_samples_split': randint(2, 20),             # Minimum number of samples to split a node
    'min_samples_leaf': randint(1, 20),              # Minimum number of samples in a leaf
    'max_features': ['auto', 'sqrt', 'log2', None],  # Number of features to consider at each split
    'bootstrap': [True, False],                      # Use bootstrapping
    'criterion': ['gini', 'entropy', 'log_loss'],    # Split quality metric
    'class_weight': [None, 'balanced', 'balanced_subsample']
}

In [55]:
model_rd_search_rf = RandomizedSearchCV(
    estimator=model_rf,             # your base model (e.g. RandomForestClassifier)
    param_distributions=param_distributions,  # the param grid using distributions
    n_iter=10,                      # number of random combinations to try
    cv=5,                           # 5-fold cross-validation
    n_jobs=-1,                      # use all processors
    scoring='f1',                   # optional: useful for classification tasks
    random_state=42                 # optional: ensures reproducibility
)

In [56]:
#model_rd_search_rf.fit(X_train_norm, y_train)

In [57]:
#joblib.dump(model_rd_search_rf, "ml_models/model_rd_search_rf.pkl")

In [58]:
model_rd_search_rf = joblib.load("ml_models/model_rd_search_rf.pkl")

In [59]:
model_rd_search_rf.best_params_

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 43,
 'max_features': 'sqrt',
 'min_samples_leaf': 4,
 'min_samples_split': 15,
 'n_estimators': 341}

In [60]:
model_rd_gridsearch = model_rd_search_rf.best_estimator_


In [61]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.80      0.81       653
           1       0.81      0.82      0.81       669

    accuracy                           0.81      1322
   macro avg       0.81      0.81      0.81      1322
weighted avg       0.81      0.81      0.81      1322



- Run Grid Search

In [62]:
param_grid = {
    'n_estimators': [300, 340, 360, 400],     # around 341
    'max_depth': [40, 43, 45, 50],            # around 43
    'min_samples_split': [10, 15, 20],        # around 15
    'min_samples_leaf': [3, 4, 5],            # around 4
    'max_features': ['sqrt'],                 # fixed since it was best
    'bootstrap': [True],                      # fixed
    'criterion': ['gini']                     # fixed
}





In [63]:
model_grid_search_rf = GridSearchCV(
    estimator=model_rf,              # your base model (e.g. RandomForestClassifier)
    param_grid=param_grid, # param grid with all possible combinations (no distributions)
    cv=5,                           # 5-fold cross-validation
    n_jobs=-1,                     # use all processors
    scoring='f1',                   # scoring metric for classification
    refit=True,                    # refit best estimator on whole dataset
    verbose=1                      # optional: to see progress
)

In [64]:
model_grid_search_rf.fit(X_train, y_train)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(cv=5,
             estimator=RandomForestClassifier(max_depth=10, random_state=42),
             n_jobs=-1,
             param_grid={'bootstrap': [True], 'criterion': ['gini'],
                         'max_depth': [40, 43, 45, 50],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [3, 4, 5],
                         'min_samples_split': [10, 15, 20],
                         'n_estimators': [300, 340, 360, 400]},
             scoring='f1', verbose=1)

In [66]:
joblib.dump(model_grid_search_rf, "ml_models/model_grid_search_rf.pkl")

['ml_models/model_grid_search_rf.pkl']

In [67]:
model_grid_search_rf = joblib.load("ml_models/model_grid_search_rf.pkl")

In [68]:
model_grid_search_rf.best_params_

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 5,
 'min_samples_split': 20,
 'n_estimators': 400}

In [69]:
model_grid_search_rf = model_grid_search_rf.best_estimator_

In [70]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.80      0.81       653
           1       0.81      0.82      0.81       669

    accuracy                           0.81      1322
   macro avg       0.81      0.81      0.81      1322
weighted avg       0.81      0.81      0.81      1322



- Evaluate your model